In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Check for GPU
device_name = tf.test.gpu_device_name()
if device_name:
    print(f"GPU available: {device_name}")
else:
    print("GPU not found. Using CPU.")


GPU available: /device:GPU:0


In [2]:
from google.colab import files
import zipfile
import os
import shutil

# Upload ZIP
uploaded = files.upload()

# Extract contents
zip_path = next(iter(uploaded))
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

# Remove sample_data folder if it exists
if os.path.exists("sample_data"):
    shutil.rmtree("sample_data")

# List directory contents
print("Extracted folders:", os.listdir())



Saving sleep_data.zip to sleep_data.zip
Extracted folders: ['.config', 'train_dataset', 'test_dataset', 'sleep_data.zip']


In [3]:
image_size = (224, 224)
batch_size = 8
num_classes = 2

train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "train_dataset",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    "test_dataset",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])


Found 292 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Set up early stopping
early_stop = EarlyStopping(
    monitor='val_loss',         # Watch validation loss
    patience=3,                 # Stop if it doesn't improve for 3 epochs
    restore_best_weights=True  # Go back to best model weights
)

# Train model
epochs = 30
model.fit(
    train_data,
    validation_data=test_data,
    epochs=epochs,
    callbacks=[early_stop]      # <<== Add callback here
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 305ms/step - accuracy: 0.7649 - loss: 0.5157 - val_accuracy: 0.9792 - val_loss: 0.3836
Epoch 2/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 1.0000 - loss: 0.1454 - val_accuracy: 0.9792 - val_loss: 0.2760
Epoch 3/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - accuracy: 1.0000 - loss: 0.0716 - val_accuracy: 0.9792 - val_loss: 0.2456
Epoch 4/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 130ms/step - accuracy: 1.0000 - loss: 0.0409 - val_accuracy: 0.9792 - val_loss: 0.2221
Epoch 5/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 1.0000 - loss: 0.0316 - val_accuracy: 0.9792 - val_loss: 0.2293
Epoch 6/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 114ms/step - accuracy: 1.0000 - loss: 0.0170 - val_accuracy: 0.9167 - val_loss: 0.2371
Epoch 7/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - accuracy: 1.0000 - loss: 0.0146 - val_accuracy: 0.9792 - val_loss: 0.2120
Epoch 8/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 1.0000 - loss: 0.0118 - val_accuracy: 0

In [6]:
# Evaluate performance on test data
loss, accuracy = model.evaluate(test_data)
print(f"Test accuracy: {accuracy:.4f}")
print(f"Test loss: {loss:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.9533 - loss: 0.1711
Test accuracy: 0.9792
Test loss: 0.1877


In [7]:
model.save("sleep_classifier_model.h5")
print("Model saved as sleep_classifier_model.h5")

from google.colab import files
files.download("sleep_classifier_model.h5")


Model saved as sleep_classifier_model.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>